In [10]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import mean_squared_error
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

In [11]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
sub_df = pd.read_csv("sample_submission.csv")

In [12]:
test_features = test_df.drop(['ID', '사고일시'], axis = 1)
sa_columns = test_features.columns
train_features = train_df[sa_columns]
train_target = train_df['ECLO']

In [13]:
cat_feature = list(train_features.dtypes[train_features.dtypes == 'object'].index)

for i in cat_feature:
  le = LabelEncoder()
  le = le.fit(train_features[i])
  train_features[i] = le.transform(train_features[i])

  for case in np.unique(test_features[i]):
    if case not in le.classes_:
      le.classes_=np.append(le.classes_, case)
  test_features[i] = le.transform(test_features[i])

<ipython-input-13-63c65da4e32a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_features[i] = le.transform(train_features[i])
<ipython-input-13-63c65da4e32a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_features[i] = le.transform(train_features[i])
<ipython-input-13-63c65da4e32a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.or

In [15]:
skf = StratifiedKFold(n_splits = 6, shuffle=True, random_state = 42)

In [16]:
hp = {
    "random_state" : 42,
    "verbose" : 0
}

model = RandomForestRegressor(**hp)
n_iter = 0
rmse_lst = []

for train_index, valid_index in skf.split(train_features, train_target):
  n_iter+=1
  train_x, valid_x = train_features.iloc[train_index], train_features.iloc[valid_index]
  train_y, valid_y = train_target.iloc[train_index], train_target.iloc[valid_index]

  model.fit(train_x, train_y)
  valid_pred = model.predict(valid_x)
  rmse = mean_squared_error(valid_y, valid_pred, squared = False)
  rmse_lst.append(rmse)
  print(f'{n_iter}번째 rmse : {rmse}')

print('-'*50)
print(f'교차 검증 rmse : {np.mean(rmse_lst)}')

prediction = model.predict(test_features)
sub_df['ECLO'] = prediction
sub_df.to_csv("LeeSubmission.csv", index=False)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=6.
  warnings.warn(


1번째 rmse : 3.3758517232146863
2번째 rmse : 3.3740583006331732
3번째 rmse : 3.407300570054935
4번째 rmse : 3.357323762975529
5번째 rmse : 3.4319689266576123
6번째 rmse : 3.3881077373469695
--------------------------------------------------
교차 검증 rmse : 3.389101836813818
